<a href="https://colab.research.google.com/github/smarsland/pots/blob/master/Shells_All_Distances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shell Distances 
## Pairwise Distances Between Shell Contours

The output of this notebook is the following three distance matrices based on the shell contours:
1. Eigenshape distances
1. SRVF Open (DP alg)
1. SRVF Closed (Path-Straightening)

### Set-Up

In [ ]:
!pip install fdasrsf==2.2.1 # Or fdasrsf==2.0.2 for closed curves
# !pip install GPy # Might be needed when using Google colab to run code.

     |████████████████████████████████| 4.0 MB 7.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for fdasrsf: filename=fdasrsf-2.3.10-cp37-cp37m-linux_x86_64.whl size=1511230 sha256=22336b6b9de2b40015f892e596146091a038401b0651d69d72bcc27f4218973c
  Stored in directory: /root/.cache/pip/wheels/b6/ee/37/ca15430451efffbcb060a9cfff9eaf027b6c2048aa2fd5d35a
Successfully built fdasrsf


#### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.spatial import procrustes
from geodesicDistances import geodDistance
import tqdm
from fdasrsf.geodesic import geod_sphere

%matplotlib inline

#### Original Contour Data & Index

In [ ]:
pth = "Shells_Final_Coords_Proc_UL.csv"
table = pd.read_csv(pth,header=None)
dfinds = pd.read_csv('Shells_Index2.csv')
indexshells = list(dfinds["Name"])

#### Re-format & Scale Contours

Re-shaping the contour dataset and scaling again...just in case.

In [ ]:
def rescale_shell(x,y):
    p = max(max(y)-min(y),max(x)-min(x))
    q = p/3
    x_ = x/q
    y_ = y/q
    mp = min(x_)+(max(x_)-min(x_))/2
    x_ = x_-mp
    mp = min(y_)+(max(y_)-min(y_))/2
    y_ = y_-mp
    return x_,y_

In [ ]:
all_points = np.zeros((len(table),int(np.shape(table)[1]/2),2))
n = len(table)
for j in range(0,n):
    coords = list(table.T[j])
    x = []
    y = []
    for i in range(0,len(coords)):
        if i%2 == 0:
            x.append(coords[i])
        else:
            y.append(coords[i])
            
    # x.append(x[0])
    # y.append(y[0])
    x,y = rescale_shell(np.array(x),np.array(y))
    all_points[j,:,0] = list(x)
    all_points[j,:,1] = list(y)

## Distance Matrix 1 - Eigenshape

Using SM's code from the Github repository, [pots](https://github.com/smarsland/pots).

In [ ]:
shells = deepcopy(all_points)

npoints = np.shape(all_points)[1]
nshells = len(table)

newshells = np.zeros(np.shape(shells))
for j in range(nshells):
    mtx1, newshells[j,:,:], _ = procrustes(shells[0,:,:],shells[j,:,:])
newshells/=np.max(newshells)

In [ ]:
# Perform PCA
shells = newshells.reshape([nshells,npoints*2])

C = np.cov(shells.T)

# Get the eigenvalues and eigenvectors
evals,evecs = np.linalg.eig(C)

# Now need to sort them into descending order
indices = np.argsort(evals)
indices = indices[::-1]
evecs = evecs[:,indices]
evals = evals[indices]
evecs = np.real(evecs)
evals = np.real(evals)
m = np.mean(shells,axis=0).reshape((npoints,2))

In [ ]:
# Screen plot of the cumulative sum of variances
cs = np.cumsum(evals)
cs /= cs[-1]
varToSave = 0.98
dims_to_use = np.where(cs>varToSave*cs[-1])[0][0]
print("Number of dims to use: "+str(dims_to_use))


Number of dims to use: 38


In [ ]:
ndims = [dims_to_use,22] # Save results from 22 dimensions too as that was the previous top performing DM for the KNN.

for ndim in ndims:
    m = np.mean(shells,axis=0)
    b = np.zeros((nshells,ndim))
    for i in range(nshells):
        P = evecs[:,:ndim]
        b[i,:] = np.dot(P.T,(shells[i,:]-m))
    dists = np.zeros((nshells,nshells))
    for i in range(nshells):
        for j in range(i,nshells):
            dists[i,j] = np.linalg.norm(b[i,:]-b[j,])
            dists[j,i] = np.linalg.norm(b[i,:]-b[j,])
    test = pd.DataFrame(dists,index=indexshells)
    test.columns = indexshells
    test.index.Name = 'Name'
    test.to_csv('Eigenshape_shells'+str(ndim)+'.csv')

## Distance Matrix 2 - SRVF _Open_

Geodesic distance using [fdasrsf](https://fdasrsf-python.readthedocs.io/en/latest/]).

In [ ]:
distances_DP = np.zeros((n,n))
errors = []

for i in tqdm.tqdm(range(0,n)):
    
    x1 = list(all_points[i,:,0])[:-1]
    y1 = list(all_points[i,:,1])[:-1]
    
    beta1 = np.column_stack([x1,y1]).T

    for j in range(i+1,n):
      x2 = list(all_points[j,:,0])[:-1]
      y2 = list(all_points[j,:,1])[:-1]
        
      beta2 = np.column_stack([x2,y2]).T
        
      try:
          d,_,_, = geod_sphere(beta2, beta1)
      except:
          try:
              d,_,_, = geod_sphere(beta2, beta1)
          except:
              print("Error for contours "+str(i)+" and "+str(j))
              errors.append([i,j])
              d = 100000
                
      distances_DP[i,j] = d
      distances_DP[j,i] = d
        
    pd.DataFrame(distances_DP).to_csv('SRVF_Open_shells_distances.csv')

print("Computed distances between "+str(n)+" contours, with "+str(len(errors))+" errors.")

100%|██████████| 235/235 [2:17:36<00:00, 35.13s/it]

Computed distances between 235 contours, with 0 errors.


In [ ]:
distancesDF = pd.DataFrame(distances_DP,index=indexshells)
distancesDF.columns = indexshells
distancesDF.index.name='Name'
distancesDF.to_csv('SRVF_Open_shells.csv')
from google.colab import files
files.download('SRVF_Open_shells.csv')


## Distance Matrix 3 - SRVF _Closed_

Using the Path-Straightening algorithm, implemented in [fdasrsf](https://fdasrsf-python.readthedocs.io/en/latest/).

In [ ]:
distances = np.zeros((n,n))
errors = []

for i in tqdm.tqdm(range(0,n)):
    x1 = list(all_points[i,:,0])
    y1 = list(all_points[i,:,1])

    for j in range(i+1,n):
        x2 = list(all_points[j,:,0])
        y2 = list(all_points[j,:,1])
                
        try:
            d,_,_,_ = geodDistance(x1,y1,x2,y2,k=2)
        except:
            try:
                d,_,_,_ = geodDistance(x2,y2,x1,y1,k=2)
            except:
                print("Error for contours "+str(i)+" and "+str(j))
                errors.append([i,j])
                d = 100000
                
        distances[i,j] = d
        distances[j,i] = d
        
    pd.DataFrame(distances).to_csv('SRVF_Closed_shells_distances.csv')

print("Computed distances between "+str(n)+" contours, with "+str(len(errors))+" errors.")

  6%|████▍                                                                      | 14/235 [1:17:31<20:27:41, 333.31s/it]

In [ ]:
distancesDF = pd.DataFrame(distances,index=indexshells)
distancesDF.columns = indexshells
distancesDF.index.name='Name'
distancesDF.to_csv('SRVF_Closed_shells.csv')